## Imports

In [111]:
np.set_printoptions(linewidth=160)
np.get_printoptions()['linewidth']

160

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import linregress

from datetime import datetime
import random
import os
import seaborn as sns
import importlib

from PIL import Image
from sklearn.cluster import KMeans

In [2]:
from skimage.segmentation import slic
from skimage.color import rgb2lab, lab2rgb
from skimage.util import img_as_ubyte
from skimage.measure import regionprops

## Start

In [206]:
import src.ArT_functions; import src.comic_book; import src.general_functions
importlib.reload(src.ArT_functions); importlib.reload(src.comic_book); importlib.reload(src.general_functions);
from src.ArT_functions import *; from src.comic_book import *; from src.general_functions import *

In [207]:
input_image_path = 'input/1705_JesseMetz1439.jpg'
output_image_path = 'output/output_image.jpg'
input_image = extract_square(input_image_path, target_size=1000)

## Move boxes

In [211]:
import numpy as np
from PIL import Image
import random


def resize_to_shape(image, target_shape):
    """
    Resize an image (numpy array) to match the target shape.
    """
    return np.array(Image.fromarray(image).resize((target_shape[1], target_shape[0]), PIL.Image.LANCZOS))


def rotate_fragment(fragment, mask, angle):
    """
    Rotate a fragment and its corresponding mask by the given angle.
    """
    fragment_img = Image.fromarray(fragment)  # Convert to PIL image
    mask_img = Image.fromarray(mask.astype(np.uint8) * 255)  # Convert mask to image
    
    # Rotate both fragment and mask
    rotated_fragment = fragment_img.rotate(angle, resample=Image.BICUBIC, expand=False)
    rotated_mask = mask_img.rotate(angle, resample=Image.BILINEAR, expand=False)

    # Convert rotated images back to numpy arrays
    rotated_fragment_arr = np.array(rotated_fragment)
    rotated_mask_arr = np.array(rotated_mask) > 128  # Threshold to get the binary mask
    
    return rotated_fragment_arr, rotated_mask_arr


def alter_image_shapes_rotation(image, num_fragments=50, offset_ratio=0.25, shape_type="circle", 
                                shape_rotation=False, fragment_rotation=False):
    """
    Alter an image by cutting it into fragments of a given shape and applying rotation to each fragment and/or shape.
    """
    arr = np.array(image)
    fragment_size = 20  # Size of each fragment block
    offset_max = 5  # Maximum offset to move fragments

    # Get dimensions
    rows, cols, _ = arr.shape

    fragment_size = cols // num_fragments
    offset_max = int(fragment_size * offset_ratio)

    # Create an output array (copy of the original image)
    fragmented_arr = arr.copy()

    # Shape definition for masks (circle, square, triangle, etc.)
    def generate_mask(shape, height, width):
        mask = np.zeros((height, width), dtype=bool)
        center_y, center_x = height // 2, width // 2
        
        if shape == "circle":
            yy, xx = np.ogrid[:height, :width]
            mask = (xx - center_x) ** 2 + (yy - center_y) ** 2 <= (min(center_x, center_y) ** 2)
        
        elif shape == "square":
            mask[height // 4:3 * height // 4, width // 4:3 * width // 4] = True
        
        elif shape == "triangle":
            for y in range(height):
                for x in range(width):
                    if x >= width // 2 - y and x <= width // 2 + y and y <= height // 2:
                        mask[y, x] = True
        
        elif shape == "pentagon":
            for y in range(height):
                for x in range(width):
                    if (x - center_x) ** 2 + (y - center_y) ** 2 <= (min(center_x, center_y) ** 2):
                        mask[y, x] = True
        
        return mask

    # Iterate over the array in blocks
    for i in range(0, rows, fragment_size):
        for j in range(0, cols, fragment_size):
            # Define the fragment boundaries
            end_row = min(i + fragment_size, rows)
            end_col = min(j + fragment_size, cols)

            # Random offset for the fragment
            offset_row = np.random.randint(-offset_max, offset_max + 1)
            offset_col = np.random.randint(-offset_max, offset_max + 1)

            # Make sure the offset doesn't move the fragment out of the image boundaries
            start_row_output = max(0, min(rows, i + offset_row))
            start_col_output = max(0, min(cols, j + offset_col))
            end_row_output = max(0, min(rows, start_row_output + (end_row - i)))
            end_col_output = max(0, min(cols, start_col_output + (end_col - j)))

            # Calculate actual fragment dimensions
            fragment_height = end_row - i
            fragment_width = end_col - j

            # Generate the mask based on the desired shape
            mask = generate_mask(shape_type, fragment_height, fragment_width)

            # Extract the fragment
            fragment = arr[i:end_row, j:end_col]

            # If shape_rotation is enabled, rotate the shape and adjust the mask accordingly
            if shape_rotation:
                angle = np.random.uniform(0, 360)
                rotated_fragment, rotated_mask = rotate_fragment(fragment, mask, angle)

                # Resize the rotated mask and fragment to match the original fragment size
                if rotated_mask.shape != fragment.shape:
                    rotated_mask_resized = resize_to_shape(rotated_mask, fragment.shape)
                else:
                    rotated_mask_resized = rotated_mask

                # Apply the rotated (and resized) mask to the rotated fragment
                destination = fragmented_arr[start_row_output:end_row_output, start_col_output:end_col_output]
                destination[rotated_mask_resized] = rotated_fragment[rotated_mask_resized]

            else:
                # If shape_rotation is disabled, use the original fragment and mask without rotation
                if mask.shape != fragment.shape:
                    mask_resized = resize_to_shape(mask, fragment.shape)
                else:
                    mask_resized = mask

                # Apply the resized mask to the original fragment
                destination = fragmented_arr[start_row_output:end_row_output, start_col_output:end_col_output]
                destination[mask_resized] = fragment[mask_resized]

            # If fragment_rotation is enabled, rotate the fragment itself
            if fragment_rotation:
                fragment = np.array(Image.fromarray(fragment).rotate(np.random.uniform(-15, 15), resample=Image.BICUBIC))

                # Apply the rotated fragment
                fragmented_arr[start_row_output:end_row_output, start_col_output:end_col_output] = fragment

    # Create the output image from the array
    fragmented_img = Image.fromarray(fragmented_arr)
    return fragmented_img


In [212]:
altered_image = alter_image_shapes_rotation(input_image, num_fragments=50, offset_ratio=0.25, 
                                            shape_type="triangle", shape_rotation=True, fragment_rotation=True)
altered_image

NameError: name 'PIL' is not defined

In [208]:
# np.random.seed(46)
# altered_image = alter_image_init(input_image, num_fragments=100, offset_ratio=.1)
# altered_image = alter_image_init_circles(input_image, num_fragments=10, offset_ratio=0.1)
# altered_image = alter_image_shapes_old(input_image, fragments_size_fraction=10, offset_ratio=0.5, shape_type='triangle')
altered_image = alter_image_shapes_rotation(input_image, fragments_size_fraction=10, offset_ratio=1, shape_type="triangle", shape_rotation=True, fragment_rotation=False)
# altered_image = alter_image_boxes_away_from_center(input_image, num_rectangles=1000)
# altered_image = alter_image_boxes(input_image, num_rectangles=10000, magnitude=.1)
# save_image_with_unique_name(altered_image, output_image_path)
altered_image

IndexError: boolean index did not match indexed array along axis 1; size of axis is 22 but size of corresponding boolean axis is 100

## Segments

In [6]:
def segment_image_with_background(img, number_of_regions, background_color=(255, 255, 255), region_size_factor=5):
    img_array = np.array(img)

    # Convert image to LAB color space for better segmentation
    img_lab = rgb2lab(img_array)

    # Calculate the maximum size per region based on the image width
    max_region_size = img.width // region_size_factor

    # Perform SLIC segmentation
    segments = slic(
        img_lab,
        n_segments=number_of_regions,
        compactness=10, 
        max_size_factor=max_region_size,
        start_label=1,
    )

    # Identify the largest region as the background
    regions = regionprops(segments)
    largest_region = max(regions, key=lambda r: r.area)
    
    # Create a new image and fill with random colors and background color
    segmented_img = np.zeros_like(img_array)
    for region in regions:
        coords = region.coords
        if region.label == largest_region.label:
            segmented_img[coords[:, 0], coords[:, 1]] = background_color  # Set background color
        else:
            random_color = np.random.randint(0, 256, size=3)  # Assign random colors to other regions
            segmented_img[coords[:, 0], coords[:, 1]] = random_color

    # Convert back to PIL image
    segmented_img_pil = Image.fromarray(img_as_ubyte(segmented_img))
    return segmented_img_pil

In [7]:
background_img = segment_image_with_background(input_image, number_of_regions=100, background_color=(255, 255, 255))
# save_image_with_unique_name(background_img, output_image_path)
# background_img

In [8]:
def segment_image_with_spatial_constraints(img, number_of_regions, region_size_factor=5, color_style="colors"):
    img_array = np.array(img)

    # Convert image to LAB color space for better color segmentation
    img_lab = rgb2lab(img_array)

    # Calculate the maximum size per region based on the image width
    max_region_size = img.width // region_size_factor
    
    # Perform SLIC segmentation
    segments = slic(
        img_lab,
        n_segments=number_of_regions,
        compactness=10,  # Balances color similarity and spatial proximity; adjust if needed
        max_size_factor=max_region_size,
        start_label=1,
    )

    # Replace each segment with a random color
    if color_style == "colors":  # Random colors
        segmented_img = np.zeros_like(img_array)
        for region in regionprops(segments):
            coords = region.coords
            random_color = np.random.randint(0, 256, size=3)  # Generate a random RGB color
            segmented_img[coords[:, 0], coords[:, 1]] = random_color
    else:  # 
        segmented_img = np.zeros_like(img_array)
        for region in regionprops(segments):
            # Get pixel indices for each region
            coords = region.coords
            # Average the color of each region
            average_color = np.mean(img_array[coords[:, 0], coords[:, 1]], axis=0)
            # Fill the region with the average color
            segmented_img[coords[:, 0], coords[:, 1]] = average_color

    # Convert back to PIL image
    segmented_img_pil = Image.fromarray(img_as_ubyte(segmented_img))
    return segmented_img_pil

In [9]:
segmented_img = segment_image_with_spatial_constraints(input_image, number_of_regions=3, color_style="colors")

# save_image_with_unique_name(segmented_img, output_image_path)
# segmented_img

In [10]:
def segment_and_average(img, number_of_regions):
    img_array = np.array(img)
    
    # Reshape the array into (number of pixels, 3) for RGB
    pixels = img_array.reshape(-1, 3)
    
    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=number_of_regions, random_state=42)
    kmeans.fit(pixels)
    
    # Get cluster centers (average colors) and labels (which cluster each pixel belongs to)
    average_colors = kmeans.cluster_centers_.astype(int)
    labels = kmeans.labels_
    
    # Replace pixel colors with the average color of their cluster
    segmented_pixels = average_colors[labels]
    
    # Reshape the array back to the original image shape
    segmented_img_array = segmented_pixels.reshape(img_array.shape)
    
    # Convert back to PIL Image
    segmented_img = Image.fromarray(segmented_img_array.astype('uint8'))
    return segmented_img

In [11]:
segmented_img = segment_and_average(input_image, number_of_regions=3)
# save_image_with_unique_name(segmented_img, output_image_path)
# segmented_img